<a href="https://colab.research.google.com/github/avocadopelvis/mental-load/blob/main/mental_load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from scipy.signal import butter, lfilter

In [44]:
def filter_signal(ecg_raw):
  def bandpass(lowcut, highcut, order=5):
          nyq = 0.5 * fs
          low = lowcut / nyq
          high = highcut / nyq
          b, a = butter(order, [low, high], btype='band')
          return b, a

  fs = 256
  lowcut = 0.1
  highcut = 45
  b,a = bandpass(lowcut,highcut)
  # ecg_raw = data_arr
  ecg_filter = lfilter(b, a, ecg_raw)
    
  return ecg_filter

def break_signal(ecg_filter):
  m = 0
  n = 3840

  #gets 20 rows from a column
  while n < 76801:
    row = ecg_filter[m:n]
    #append to final
    final.append(row)
    m += 3840
    n += 3840
  
  return final

In [67]:
# final df
final_df = pd.DataFrame()

# subjects 002 & 006 are not used here since both of their resting ecg dataset does not contaain 76800 samples
folders = ["002", "004", "005", "006", "008", "010", "011", "012", "013", "014", "015", "016", "017", "018", "019", "020", "021", "022", "023", "024", "025"]

# iterate through the folder of each subject
for folder in folders:
  # LOAD DATA
  data = pd.read_csv(f"/content/drive/MyDrive/MENTAL-LOAD-DATASET/{folder}/inf_ecg.csv")
  rest_data = pd.read_csv(f"/content/drive/MyDrive/MENTAL-LOAD-DATASET/{folder}/inf_resting.csv") 

  # drop columns in data
  data = data.drop(["Trial 2:2back", "Trial 4:2back"], axis = 1)

  # get resting ecg column
  rest_data = rest_data["Resting_ECG"]

  # add resting ecg column to data
  data = pd.concat([rest_data, data], axis = 1)

  # intitalize final
  final = []

  #iterate through data's columns i.e ["Resting_ECG", "Trial 1:0back", "Trial 3:3back", "Trial 5:3back", "Trial 6:0back"]
  for col in data.columns: 
    #take a single column
    data_col = data[col]

    # convert column (df) to numpy array
    # data_arr = data_col.to_numpy() # might not be necessary 

    # FILTER THE SIGNAL
    # def bandpass(lowcut, highcut, order=5):
    #     nyq = 0.5 * fs
    #     low = lowcut / nyq
    #     high = highcut / nyq
    #     b, a = butter(order, [low, high], btype='band')
    #     return b, a

    # fs = 256
    # lowcut = 0.1
    # highcut = 45
    # b,a = bandpass(lowcut,highcut)
    # ecg_raw = data_col
    # ecg_filter = lfilter(b,a,ecg_raw)
    filter_signal(data_col)

    # BREAK THE SIGNAL
    # m = 0
    # n = 3840

    # #gets 20 rows from a column
    # while n < 76801:
    #   row = ecg_filter[m:n]
    #   #append to final
    #   final.append(row)
    #   m += 3840
    #   n += 3840
    break_signal(ecg_filter)

  #final should now have 100 rows from all 5 columns

  # load final as a df
  df = pd.DataFrame(final)
  # assign labels
  df = df.assign(label = 20*[0] + 20*[1] + 20*[2] + 20*[2] + 20*[1])

  #append df to final_df
  final_df = final_df.append(df)

In [68]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,3831,3832,3833,3834,3835,3836,3837,3838,3839,label
0,-0.001914,-0.014633,-0.051734,-0.113836,-0.178491,-0.219520,-0.232475,-0.233461,-0.237069,-0.242072,...,-0.010367,-0.017412,-0.019947,-0.011166,0.008859,0.030297,0.040837,0.035396,0.019893,0
1,0.006965,0.006299,0.017595,0.033019,0.044961,0.050178,0.049475,0.046176,0.043493,0.042296,...,0.029044,0.028440,0.032293,0.039771,0.047832,0.053091,0.053884,0.052654,0.055029,0
2,0.065412,0.083969,0.106032,0.123612,0.131067,0.130347,0.128638,0.132428,0.144718,0.164065,...,0.056806,0.022407,0.006346,0.016731,0.040341,0.055061,0.049953,0.031885,0.015553,0
3,0.010116,0.014389,0.021955,0.028266,0.032252,0.034236,0.035407,0.037704,0.041318,0.042700,...,0.117929,0.142543,0.153108,0.147799,0.139133,0.137886,0.143688,0.150131,0.153582,0
4,0.155186,0.157407,0.162049,0.170433,0.181727,0.191721,0.197128,0.200121,0.204499,0.209229,...,-0.325091,-0.332651,-0.334824,-0.336602,-0.341318,-0.346272,-0.346752,-0.341391,-0.332376,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.131565,-0.136414,-0.139860,-0.132463,-0.113560,-0.094689,-0.088723,-0.095477,-0.099772,-0.086283,...,-0.105362,-0.102184,-0.089493,-0.075865,-0.068164,-0.067680,-0.068634,-0.062618,-0.048078,1
96,-0.034381,-0.032947,-0.044263,-0.057791,-0.064437,-0.063073,-0.055331,-0.042823,-0.030440,-0.022912,...,-0.052154,-0.052397,-0.053673,-0.050650,-0.042439,-0.033684,-0.029129,-0.029195,-0.031468,1
97,-0.032855,-0.030765,-0.025271,-0.018512,-0.011355,-0.004243,-0.000105,-0.001208,-0.004484,-0.004197,...,-0.182481,-0.195720,-0.195329,-0.201010,-0.220380,-0.243190,-0.255585,-0.255883,-0.252987,1
98,-0.253310,-0.255877,-0.257402,-0.256288,-0.252513,-0.246039,-0.236466,-0.225157,-0.216629,-0.214664,...,-0.024017,-0.017511,-0.012621,-0.012055,-0.013178,-0.011728,-0.007674,-0.004027,-0.002103,1


In [66]:
# drop rows containing NaN values 
# two rows contain NaN values
final_df = final_df.dropna()
final_df

,0,1,2,3,4,5,6,7,8,9,...,3831,3832,3833,3834,3835,3836,3837,3838,3839,label
0,-0.001914,-0.014633,-0.051734,-0.113836,-0.178491,-0.219520,-0.232475,-0.233461,-0.237069,-0.242072,...,-0.010367,-0.017412,-0.019947,-0.011166,0.008859,0.030297,0.040837,0.035396,0.019893,0
1,0.006965,0.006299,0.017595,0.033019,0.044961,0.050178,0.049475,0.046176,0.043493,0.042296,...,0.029044,0.028440,0.032293,0.039771,0.047832,0.053091,0.053884,0.052654,0.055029,0
2,0.065412,0.083969,0.106032,0.123612,0.131067,0.130347,0.128638,0.132428,0.144718,0.164065,...,0.056806,0.022407,0.006346,0.016731,0.040341,0.055061,0.049953,0.031885,0.015553,0
3,0.010116,0.014389,0.021955,0.028266,0.032252,0.034236,0.035407,0.037704,0.041318,0.042700,...,0.117929,0.142543,0.153108,0.147799,0.139133,0.137886,0.143688,0.150131,0.153582,0
4,0.155186,0.157407,0.162049,0.170433,0.181727,0.191721,0.197128,0.200121,0.204499,0.209229,...,-0.325091,-0.332651,-0.334824,-0.336602,-0.341318,-0.346272,-0.346752,-0.341391,-0.332376,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.131565,-0.136414,-0.139860,-0.132463,-0.113560,-0.094689,-0.088723,-0.095477,-0.099772,-0.086283,...,-0.105362,-0.102184,-0.089493,-0.075865,-0.068164,-0.067680,-0.068634,-0.062618,-0.048078,1
96,-0.034381,-0.032947,-0.044263,-0.057791,-0.064437,-0.063073,-0.055331,-0.042823,-0.030440,-0.022912,...,-0.052154,-0.052397,-0.053673,-0.050650,-0.042439,-0.033684,-0.029129,-0.029195,-0.031468,1
97,-0.032855,-0.030765,-0.025271,-0.018512,-0.011355,-0.004243,-0.000105,-0.001208,-0.004484,-0.004197,...,-0.182481,-0.195720,-0.195329,-0.201010,-0.220380,-0.243190,-0.255585,-0.255883,-0.252987,1
98,-0.253310,-0.255877,-0.257402,-0.256288,-0.252513,-0.246039,-0.236466,-0.225157,-0.216629,-0.214664,...,-0.024017,-0.017511,-0.012621,-0.012055,-0.013178,-0.011728,-0.007674,-0.004027,-0.002103,1


In [54]:
final_df[final_df.isna().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,3831,3832,3833,3834,3835,3836,3837,3838,3839,label


# FOR SUBJECTS 002 & 006

In [57]:
# empty = pd.DataFrame()

# folders = ["002", "006"] 

# # iterate through the folder of each subject
# for folder in folders:
#   # LOAD DATA
#   data = pd.read_csv(f"/content/drive/MyDrive/MENTAL-LOAD-DATASET/{folder}/inf_ecg.csv")
#   rest_data = pd.read_csv(f"/content/drive/MyDrive/MENTAL-LOAD-DATASET/{folder}/inf_resting.csv") 

#   # drop columns in data
#   data = data.drop(["Trial 2:2back", "Trial 4:2back"], axis = 1)

#   # get resting ecg column
#   rest_data = rest_data["Resting_ECG"]

#   # convert it into a numpy array
#   rest_data = rest_data.to_numpy()

#   # insert resting ecg column to 0th position
#   data.insert(0, "Resting_ECG", rest_data)

#   # intitalize final
#   final = []
#   #iterate through data's columns i.e ["Resting_ECG", "Trial 1:0back", "Trial 3:3back", "Trial 5:3back", "Trial 6:0back"]
#   for col in data.columns: # 5 times
#     #take a single column
#     data_col = data[col]

#     # convert column (df) to numpy array
#     data_arr = data_col.to_numpy()

#     # FILTER THE SIGNAL
#     def bandpass(lowcut, highcut, order=5):
#         nyq = 0.5 * fs
#         low = lowcut / nyq
#         high = highcut / nyq
#         b, a = butter(order, [low, high], btype='band')
#         return b, a

#     fs = 256
#     lowcut = 0.1
#     highcut = 45
#     b,a = bandpass(lowcut,highcut)
#     ecg_raw = data_arr
#     ecg_filter = lfilter(b,a,ecg_raw)

#     # BREAK THE SIGNAL
#     m = 0
#     n = 3840

#     #gets 20 rows from a column
#     while n < 76801:
#       row = ecg_filter[m:n]
#       #append to final
#       final.append(row)
#       m += 3840
#       n += 3840

#   #final should now have 99 rows from all 5 columns

#   # load final as a df
#   df = pd.DataFrame(final)
#   # assign labels
#   df = df.assign(label = 20*[0] + 20*[1] + 20*[2] + 20*[2] + 20*[1])

#   #append df to final_df
#   final_df = final_df.append(df)

In [ ]:
# convert df to csv
df.to_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")

# TEST

In [ ]:
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# cols = ["Resting_ECG", "Trial 1:0back", "Trial 3:3back", "Trial 5:3back", "Trial 6:0back"]

# convert it into a numpy array
# rest_data = rest_data.to_numpy() 

# insert resting ecg column to 0th position
# data.insert(0, "Resting_ECG", rest_data)

In [43]:
final = []
data = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/002/inf_ecg.csv")
rest_data = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/002/inf_resting.csv") 
rest_data = rest_data["Resting_ECG"]
# rest_data = rest_data.to_numpy()
# FILTER THE SIGNAL
from scipy.signal import butter, lfilter

def bandpass(lowcut, highcut, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

fs = 256
lowcut = 0.1
highcut = 45
b,a = bandpass(lowcut,highcut)
ecg_raw = rest_data
ecg_filter = lfilter(b,a,ecg_raw)

# BREAK THE SIGNAL
m = 0
n = 3840

while n < 76801:
  row = ecg_filter[m:n]
  #append to final
  final.append(row)
  m += 3840
  n += 3840

test = pd.DataFrame(final)
test  

,0,1,2,3,4,5,6,7,8,9,...,3830,3831,3832,3833,3834,3835,3836,3837,3838,3839
0,-0.000937,-0.007203,-0.025761,-0.057455,-0.090132,-0.106145,-0.099081,-0.081342,-0.071394,-0.073434,...,-0.178658,-0.195488,-0.209389,-0.221418,-0.231474,-0.239308,-0.245136,-0.249153,-0.251306,-0.251726
1,-0.250747,-0.248100,-0.242925,-0.235485,-0.228200,-0.223662,-0.221579,-0.218735,-0.212500,-0.203598,...,0.076493,0.053954,0.035626,0.022383,0.010377,-0.002666,-0.014822,-0.023016,-0.026749,-0.028513
2,-0.031275,-0.035827,-0.040436,-0.042807,-0.042064,-0.039083,-0.035664,-0.033251,-0.031587,-0.028869,...,0.273793,0.242360,0.206749,0.176540,0.166375,0.182852,0.217199,0.252381,0.275658,0.284019
3,0.282774,0.282530,0.291160,0.303156,0.302636,0.282632,0.257437,0.250007,0.268365,0.298846,...,-0.000067,-0.000216,0.002085,0.007211,0.012925,0.016449,0.016999,0.016249,0.017634,0.027781
4,0.058582,0.123291,0.227564,0.364535,0.516889,0.659293,0.750784,0.724379,0.508810,0.090546,...,-0.286236,-0.283535,-0.277764,-0.270363,-0.263406,-0.257478,-0.252091,-0.246567,-0.240262,-0.233057
5,-0.225518,-0.218169,-0.211099,-0.204293,-0.197885,-0.192182,-0.187498,-0.183599,-0.179399,-0.173690,...,-0.318207,-0.324854,-0.327979,-0.330363,-0.332934,-0.333844,-0.332352,-0.330019,-0.327779,-0.324064
6,-0.316530,-0.305532,-0.294823,-0.287248,-0.281169,-0.273232,-0.263015,-0.252512,-0.242541,-0.232788,...,-0.004131,-0.003408,-0.003406,-0.002792,-0.000562,0.003430,0.008254,0.013007,0.018175,0.024719
7,0.032985,0.043525,0.056990,0.072166,0.085607,0.094025,0.096965,0.097194,0.097978,0.099438,...,0.115239,0.205423,0.332268,0.478832,0.617063,0.708452,0.699138,0.526037,0.160109,-0.337971
8,-0.817788,-1.115783,-1.150987,-0.963705,-0.670359,-0.380501,-0.148517,0.016880,0.122741,0.179302,...,-0.070337,-0.065515,-0.060324,-0.055998,-0.052548,-0.049274,-0.045829,-0.042264,-0.038625,-0.034797
9,-0.030619,-0.026329,-0.022715,-0.020175,-0.017925,-0.015033,-0.011671,-0.008503,-0.005493,-0.001955,...,-0.102102,-0.097272,-0.091340,-0.084716,-0.078684,-0.074085,-0.070946,-0.068614,-0.066071,-0.062516


In [55]:
final1 = []
data = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/006/inf_ecg.csv")
rest_data = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/006/inf_resting.csv") 

# drop columns in data
data = data.drop(["Trial 2:2back", "Trial 4:2back"], axis = 1)

# get resting ecg column
rest_data = rest_data["Resting_ECG"]

data = pd.concat([rest_data, data], axis = 1)

#iterate through data's columns i.e ["Resting_ECG", "Trial 1:0back", "Trial 3:3back", "Trial 5:3back", "Trial 6:0back"]
for col in data.columns: 
  #take a single column
  data_col = data[col]

  # convert column (df) to numpy array
  # data_arr = data_col.to_numpy()

  # FILTER THE SIGNAL
  def bandpass(lowcut, highcut, order=5):
      nyq = 0.5 * fs
      low = lowcut / nyq
      high = highcut / nyq
      b, a = butter(order, [low, high], btype='band')
      return b, a

  fs = 256
  lowcut = 0.1
  highcut = 45
  b,a = bandpass(lowcut,highcut)
  ecg_raw = data_col
  ecg_filter = lfilter(b,a,ecg_raw)

  # BREAK THE SIGNAL
  m = 0
  n = 3840

  #gets 20 rows from a column
  while n < 76801:
    row = ecg_filter[m:n]
    #append to final
    final1.append(row)
    m += 3840
    n += 3840

test = pd.DataFrame(final1)
test

,0,1,2,3,4,5,6,7,8,9,...,3830,3831,3832,3833,3834,3835,3836,3837,3838,3839
0,0.005338,0.039376,0.132762,0.274023,0.393179,0.425829,0.376896,0.305738,0.263522,0.253452,...,-0.116070,-0.112271,-0.110335,-0.108439,-0.105392,-0.101092,-0.096227,-0.091834,-0.088565,-0.085430
1,-0.079849,-0.070573,-0.060169,-0.052793,-0.049452,-0.047025,-0.042110,-0.034512,-0.026520,-0.019725,...,-0.980385,-0.705301,-0.411601,-0.181839,-0.026166,0.089794,0.191507,0.273180,0.318212,0.327454
2,0.320139,0.313659,0.312270,0.313219,0.314102,0.312750,0.307319,0.299693,0.294333,0.291748,...,0.168656,0.166668,0.166477,0.165735,0.163246,0.161324,0.162128,0.163512,0.161406,0.155800
3,0.150947,0.149115,0.147791,0.144177,0.138792,0.132974,0.126229,0.118106,0.109766,0.101687,...,-0.168691,-0.164332,-0.159921,-0.155781,-0.152152,-0.148902,-0.145554,-0.141810,-0.138123,-0.135368
4,-0.133851,-0.132915,-0.131617,-0.129692,-0.127819,-0.126817,-0.126552,-0.125910,-0.123930,-0.120720,...,0.156072,0.145373,0.133080,0.118339,0.102671,0.087299,0.070905,0.051889,0.030990,0.009704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.146591,-0.151036,-0.152722,-0.152064,-0.146498,-0.132501,-0.111957,-0.094047,-0.088655,-0.098194,...,0.081937,0.156701,0.267908,0.407965,0.548059,0.626122,0.577249,0.386127,0.099885,-0.220370
96,-0.544910,-0.850844,-1.072141,-1.119147,-0.960380,-0.666197,-0.360517,-0.131030,0.015225,0.119370,...,0.010200,0.007604,-0.000118,-0.000662,0.010495,0.023760,0.025846,0.014019,-0.000802,-0.004707
97,0.006437,0.024999,0.040164,0.047004,0.048519,0.050070,0.052258,0.051418,0.047326,0.045657,...,0.302729,0.291439,0.281250,0.269556,0.256612,0.247525,0.246285,0.248536,0.245200,0.234096
98,0.223014,0.219347,0.221263,0.222824,0.224403,0.231721,0.244799,0.254549,0.252535,0.239849,...,-0.331947,-0.329637,-0.327387,-0.316630,-0.296419,-0.274369,-0.260859,-0.260662,-0.268037,-0.271755


In [56]:
test = test.dropna()
test

,0,1,2,3,4,5,6,7,8,9,...,3830,3831,3832,3833,3834,3835,3836,3837,3838,3839
0,0.005338,0.039376,0.132762,0.274023,0.393179,0.425829,0.376896,0.305738,0.263522,0.253452,...,-0.116070,-0.112271,-0.110335,-0.108439,-0.105392,-0.101092,-0.096227,-0.091834,-0.088565,-0.085430
1,-0.079849,-0.070573,-0.060169,-0.052793,-0.049452,-0.047025,-0.042110,-0.034512,-0.026520,-0.019725,...,-0.980385,-0.705301,-0.411601,-0.181839,-0.026166,0.089794,0.191507,0.273180,0.318212,0.327454
2,0.320139,0.313659,0.312270,0.313219,0.314102,0.312750,0.307319,0.299693,0.294333,0.291748,...,0.168656,0.166668,0.166477,0.165735,0.163246,0.161324,0.162128,0.163512,0.161406,0.155800
3,0.150947,0.149115,0.147791,0.144177,0.138792,0.132974,0.126229,0.118106,0.109766,0.101687,...,-0.168691,-0.164332,-0.159921,-0.155781,-0.152152,-0.148902,-0.145554,-0.141810,-0.138123,-0.135368
4,-0.133851,-0.132915,-0.131617,-0.129692,-0.127819,-0.126817,-0.126552,-0.125910,-0.123930,-0.120720,...,0.156072,0.145373,0.133080,0.118339,0.102671,0.087299,0.070905,0.051889,0.030990,0.009704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.146591,-0.151036,-0.152722,-0.152064,-0.146498,-0.132501,-0.111957,-0.094047,-0.088655,-0.098194,...,0.081937,0.156701,0.267908,0.407965,0.548059,0.626122,0.577249,0.386127,0.099885,-0.220370
96,-0.544910,-0.850844,-1.072141,-1.119147,-0.960380,-0.666197,-0.360517,-0.131030,0.015225,0.119370,...,0.010200,0.007604,-0.000118,-0.000662,0.010495,0.023760,0.025846,0.014019,-0.000802,-0.004707
97,0.006437,0.024999,0.040164,0.047004,0.048519,0.050070,0.052258,0.051418,0.047326,0.045657,...,0.302729,0.291439,0.281250,0.269556,0.256612,0.247525,0.246285,0.248536,0.245200,0.234096
98,0.223014,0.219347,0.221263,0.222824,0.224403,0.231721,0.244799,0.254549,0.252535,0.239849,...,-0.331947,-0.329637,-0.327387,-0.316630,-0.296419,-0.274369,-0.260859,-0.260662,-0.268037,-0.271755


In [40]:
data = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/004/inf_ecg.csv")
rest_data = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/004/inf_resting.csv") 

# drop columns in data
data = data.drop(["Trial 2:2back", "Trial 4:2back"], axis = 1)

# get resting ecg column
rest_data = rest_data["Resting_ECG"]



data

,Resting_ECG,Trial 1:0back,Trial 3:3back,Trial 5:3back,Trial 6:0back
0,-0.105,-0.084,0.213,0.122,0.207
1,-0.158,-0.066,0.213,0.282,0.203
2,-0.171,-0.038,0.224,0.428,0.210
3,-0.165,-0.038,0.220,0.451,0.215
4,-0.131,-0.061,0.223,0.318,0.220
...,...,...,...,...,...
76795,0.078,-0.055,-0.116,-0.082,0.220
76796,0.074,-0.063,-0.132,-0.085,0.219
76797,0.070,-0.068,-0.138,-0.083,0.225
76798,0.063,-0.073,-0.132,-0.079,0.218
